In [14]:
import numpy as np 
from numpy import linalg as LA
from ncon import ncon
from scipy.sparse.linalg import LinearOperator, eigs

In [16]:
d = 2; 
chi = 5; 
A = np.random.rand(chi, d, chi)
B = np.random.rand(chi, d, chi)
sAB = np.ones(chi) / np.sqrt(chi) 
sBA = np.ones(chi) / np.sqrt(chi)

In [17]:
""" Contract infinite MPS from the left for environment tensor sigBA """
sigBA = np.random.rand(chi, chi)  
tol = 1e-3
# define the tensor network
tensors = [np.diag(sBA), np.diag(sBA), A, A.conj(), np.diag(sAB),
           np.diag(sAB), B, B.conj()]
labels = [[1,2],[1,3],[2,4],[3,5,6],[4,5,7],[6,8],[7,9],[8,10,-1],[9,10,-2]]
for j in range(100):
    sigBA_new = ncon([sigBA, *tensors],labels)
    sigBA_new = sigBA_new / np.trace(sigBA_new)
    if LA.norm(sigBA - sigBA_new) < tol:
        print("Success")
        break
    else:
        print("iter: {}, diff: {}".format(j, LA.norm(sigBA - sigBA_new)))
        sigBA = sigBA_new

iter: 0, diff: 2.058206855790333
iter: 1, diff: 0.005526188983734901
Success


In [18]:
def left_contract_MPS(sigBA, sBA, A, sAB,B):
    D_BA = A.shape[0]
    if sigBA.shape[0] == D_BA:
        v0 = sigBA.reshape(np.prod(sigBA.shape))
    else:
        v0 = (np.eye(D_BA) / D_BA).reshape(D_BA**2)
    tensors = [np.diag(sBA), np.diag(sBA), A, A.conj(), np.diag(sAB),
             np.diag(sAB), B, B.conj()]
    labels = [[1, 2], [1, 3], [2, 4], [3, 5, 6], [4, 5, 7], [6, 8], [7, 9],
              [8, 10, -1], [9, 10, -2]]
      # define function for boundary contraction and pass to eigs
    def left_iter(sigBA):
        return ncon([sigBA.reshape([D_BA,D_BA]), *tensors],
                   labels).reshape([D_BA**2,1])
    
        
    Dtemp, sigBA = eigs(LinearOperator((D_BA**2,D_BA**2), matvec=left_iter),
                       k=1,which='LM',v0=v0,tol=1e-10)
    
    if np.isrealobj(A):
        sigBA = np.real(sigBA)
    sigBA = sigBA.reshape(D_BA,D_BA)
    sigBA = 0.5*(sigBA + np.conj(sigBA.T))
    sigBA = sigBA / np.trace(sigBA)
    
    sigAB = ncon([sigBA, np.diag(sBA), np.diag(sBA), A, np.conj(A)],
                 [[1, 2], [1, 3], [2, 4], [3, 5, -1], [4, 5, -2]])
    sigAB = sigAB / np.trace(sigAB)
    return sigBA, sigAB